## A semi-automated testing and optimising example for the MCTS' "playouts" and "c" parameters ##

-----

# Part 1 - setup and classes #

In [2]:
### IMPORTS

from mctsNode import mctsNode
import mctsSearch_Basic as mcts
import mctsSearch_Basic_timelimited as mctstimed

from time import monotonic
from copy import deepcopy

• Time aspect - special editing to mcts + controlled for each search from the agent <br>
• Breadth - dynamic needs implementing in mcts + some external way to affect it easily <br>
• C - currently testable straight away <br>
• Need a book of opening moves <br>

set up agent 1 args <br>
set up agent 2 args 


While N iterations <br>
    play the two agents

In [3]:
### SIMPLE MCTS AGENTS

# Global Parameter for local testing - iterations or time
iterations = 1000

# Simple Agent - static
# Can be used to optimise **playouts**, and **c**

class simplemctsAgent:
    def __init__(self, _playouts, _c):
        
        ## set up simple params 
        # like c, playouts number, iterations must be same for all...
        self.playouts = _playouts
        self.c = _c
        
    def makeMove(self, board):
        root = mctsNode(None, board, None)
        child = mcts.mctsSearch(root,iterations,self.playouts,self.c)
        move = child.getMove()
        return move

    
# same but fixed time allocated

class simple_mcts_timelimited_Agent:
    def __init__(self, _playouts, _c, _time):
        
        ## set up simple params 
        # like c, playouts number, iterations must be same for all...
        self.playouts = _playouts
        self.c = _c
        self.time = _time
        
    def makeMove(self, board):
        root = mctsNode(None, board, None)
        child = mctstimed.mctsSearch(root,self.time,self.playouts,self.c)
        move = child.getMove()
        return move
        
        

In [4]:
### BASIC GAME CLASS IMPLEMENTATION

class Game:
    
    def __init__(self, _agent1, _agent2, _reps):
        self.agent1 = _agent1
        self.agent2 = _agent2
        self.reps = _reps
        self.stat = None
        
    def run(self):
        wins = 0
        for i in range(self.reps):
            wins += self._play(i%2)
#             print(wins)
        self.stat = wins / self.reps
        
    def win_rate(self):
        return self.stat
    
    def _play(self, first):
        board = [
            [0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],
        ]
        player1 = first
        while True:
            if player1:
                move = self.agent1.makeMove(board)
                board[move[0]][move[1]] = 1
                if mcts._winner(board):
                    return 1
                
            
            else:
                board = self._opposite_board(board)
                move = self.agent2.makeMove(board)
                board[move[0]][move[1]] = 1
                if mcts._winner(board):
                    return 0
                board = self._opposite_board(board)
                
            player1 = not player1
            
        return None    
        
        
    def _opposite_board(self, board):
        opp = deepcopy(board)
        #rotate along diagonal
        for i in range(11):
            for j in range(i,11):
                opp[i][j], opp[j][i] = opp[j][i], opp[i][j]
        # and change -1 and 1        
        for i in range(11):
            for j in range(11):
                opp[i][j]*= -1
        return opp
        
            

-----------

# Part 2 - Basic test example #

In [8]:
### IGNORE
### JUST A CHECK THAT MY CODE WORKS

# start = monotonic()
# benchmark test - 1 vs 20 default playouts, c = 0.5, 1 sec per mcts
# simple_mcts_timelimited_Agent(1, 0.08, 1)
# simple_mcts_timelimited_Agent(3, 0.1, 5)
game = Game(simple_mcts_timelimited_Agent(1, 0.5, 1), simple_mcts_timelimited_Agent(1, 0.5, 1), 3)
game.run()
print(game.win_rate())
# end = monotonic()
# print("Time elapsed during matches:", end - start)


#===============
#  0.6
#  Time elapsed during matches: 340.4184336870003
#===============
# Result for same agents was 0.6 proving that my testing method is all correct
# (I was afraid it might somehow be bugging and only return 1 or 0)
#===============

0.3333333333333333


In [45]:
### BASIC TEST - 1 MATCH BETWEEN 2 AGENTS

# track time (not needed for the Game)
start = monotonic()

# benchmark test 
# 1 vs 20 default playouts with c = 0.5 and 1 sec per mcts
agenta = simple_mcts_timelimited_Agent(1, 0.5, 1)
agentb = simple_mcts_timelimited_Agent(20, 0.5, 1)
# and only 1 match
match_reruns = 1

# set up game, run, and get win_rate for agent1
game = Game(agenta, agentb, match_reruns)
game.run()
print(game.win_rate())

# print seconds for that game
end = monotonic()
print("Time elapsed during matches:", end - start)
# about 1 minute

1
1.0
Time elapsed during matches: 67.71332813499976


Above output: <br>
1.0 - Win_rate for agent1<br>
Time elapsed during matches: 72.069467774 <br>


Let's do more of a tournament thing
- lets keep the mcts to a minute first
- 10 plays per pair ~12 minutes / pair
- in total 21 unique pairs to play => ~4 hours damn
- => will keep it to 3 pairs per set and gain heuristics from that



-----------

# Part 3 - Analyze and optimise 'playouts' parameter #

### 3.1 - Fixed time for msct - 1 sec

In [46]:
time = 1  #expect about 120secs / 2 mins per game

agenta = simple_mcts_timelimited_Agent(1, 0.5, time)
agentb = simple_mcts_timelimited_Agent(2, 0.5, time)
agentc = simple_mcts_timelimited_Agent(3, 0.5, time)
agentd = simple_mcts_timelimited_Agent(4, 0.5, time)
agente = simple_mcts_timelimited_Agent(5, 0.5, time)
agentf = simple_mcts_timelimited_Agent(10, 0.5, time)
agentg = simple_mcts_timelimited_Agent(20, 0.5, time)

In [24]:
results = []

In [25]:
#cycle 1 - default playouts numbers 1 to 4

players = [(agenta,agentb), (agenta,agentc), (agenta,agentd)]

for pair in players:
    start = monotonic()
    game = Game(pair[0], pair[1], 10)
    game.run()
    print(game.win_rate())
    end = monotonic()
    minutes = int((end - start)/60)
    print("Time elapsed during the 10 plays (in minutes):", minutes)
    results.append((game.win_rate(),minutes))

    
print(results)

1.0
Time elapsed during the 10 plays (in minutes): 10
1.0
Time elapsed during the 10 plays (in minutes): 11
1.0
Time elapsed during the 10 plays (in minutes): 10
[(1.0, 10), (1.0, 11), (1.0, 10)]


<b> Results from above test - clearly with a 1 minute per tree, it's the playouts=1 thats best. </b> 

1.0 <br>
Time elapsed during the 10 plays (in minutes): 10 <br>
1.0 <br>
Time elapsed during the 10 plays (in minutes): 11 <br>
1.0 <br>
Time elapsed during the 10 plays (in minutes): 10 <br>
[(1.0, 10), (1.0, 11), (1.0, 10)]  <br>

<b> So with only 1 sec per tree there is a clear winner - playout = 1 </b> 

-> this is probably because the tree doesn't get big enough to be worth the extra time we put into better resolution leafs data (1/0 vs 1/0.66/0.33/0 for 3)


### 3.2 - time for mcts - 5 sec


<b> Test 2 - more time per search </b> <br>
- let's see if anything changes if we make time = 5 seconds per search
- we are going to run a single 10 iteration on 1 vs 2 and let's see 

In [39]:
time = 5  #expect about 600secs / 5 mins per game (max) - more like 3

agenta = simple_mcts_timelimited_Agent(1, 0.5, time)
agentb = simple_mcts_timelimited_Agent(2, 0.5, time)

start = monotonic()
# benchmark test - 1 vs 20 default playouts, c = 0.5, 1 sec per mcts
game = Game(agenta, agentb, 10)
game.run()
print(game.win_rate())
end = monotonic()
print("Time elapsed during matches:", end - start)

0.3
Time elapsed during matches: 1793.2188330210001


0.3 <br>
Time elapsed during matches: 1793.2188330210001 ~30min

Exciting! <br>
With longer time per search - 5 second - it turns out that agent 2 is better than agent 1 (2 playouts rather than 1) 

<b> With time = 5sec playouts = 2 is better than playouts = 1 </b>

Therefore now we have to compare 2 and 3

In [40]:
time = 5  #expect about 600secs / 5 mins per game (max) - more like 3

agenta = simple_mcts_timelimited_Agent(2, 0.5, time)
agentb = simple_mcts_timelimited_Agent(3, 0.5, time)

start = monotonic()
# benchmark test - 1 vs 20 default playouts, c = 0.5, 1 sec per mcts
game = Game(agenta, agentb, 10)
game.run()
print(game.win_rate())
end = monotonic()
print("Time elapsed during matches:", end - start)

0.4
Time elapsed during matches: 2090.7558067339996


0.4
Time elapsed during matches: 2090.7558067339996 ~35min

With time = 5sec in fact playouts = 3 is better than =2
Furthermore note time was more i.e. the two agents had to play further down the game (i.e. made fewer mistakes)

### Conclusions of part 3 ###
1. Generally too many playouts seem to beat the purpose of mcts
2. A number of playouts slightly bigger than 1 might improve playing quality
3. The exact optimal number of playouts depends on mcts time (x processing power) => so it needs to be tested in regards to the final machine capabilities <br>
-- an intuition: this is probably the case because in the more constrained case (1 sec) we don't have enough time to take advantage of the higher resolution of new Nodes' estimated values <br>
-- question: can we gain the same insight but with n-iterations instead of n-seconds per MCTS, and then just find how n-iterations relate to the uni machine's speed

-----------

# Part 4 - attempt at analyzing and optimising 'c' parameter #

### 4.1 Testing our intuition, and c with resolution .1 ###

<b> I expect the above notes and conclusions to be roughly the trend for c too </b> <br>
To slightly bias towards exploration the more time the mcts has, and bias toward exploitation the less time it has.

Run 1 - 1 second per move

In [41]:
time = 1  #expect about 120secs / 2 mins per game

agent1 = simple_mcts_timelimited_Agent(1, 0.1, time)
agent2 = simple_mcts_timelimited_Agent(1, 0.2, time)
agent3 = simple_mcts_timelimited_Agent(1, 0.3, time)
agent4 = simple_mcts_timelimited_Agent(1, 0.4, time)
agent5 = simple_mcts_timelimited_Agent(1, 0.5, time)
agent6 = simple_mcts_timelimited_Agent(1, 0.6, time)
agent7 = simple_mcts_timelimited_Agent(1, 0.7, time)
agent8 = simple_mcts_timelimited_Agent(1, 0.8, time)

players = [(agent1,agent2), (agent3,agent4), (agent5,agent6), (agent7,agent8)]
results = []

for pair in players:
    start = monotonic()
    game = Game(pair[0], pair[1], 10)
    game.run()
    print(game.win_rate())
    end = monotonic()
    minutes = int((end - start)/60)
    print("Time elapsed during the 10 plays (in minutes):", minutes)
    results.append((game.win_rate(),minutes))

    
print(results)

0.9
Time elapsed during the 10 plays (in minutes): 8
0.7
Time elapsed during the 10 plays (in minutes): 10
0.6
Time elapsed during the 10 plays (in minutes): 11
0.4
Time elapsed during the 10 plays (in minutes): 11
[(1.0, 10), (1.0, 11), (1.0, 10), (0.9, 8), (0.7, 10), (0.6, 11), (0.4, 11)]


0.9 <br>
Time elapsed during the 10 plays (in minutes): 8 <br>
0.7 <br>
Time elapsed during the 10 plays (in minutes): 10 <br>
0.6 <br>
Time elapsed during the 10 plays (in minutes): 11 <br>
0.4 <br>
Time elapsed during the 10 plays (in minutes): 11 <br>

so results: <br>
1 vs 2 : 9:1  |  3 vs 4 : 7:3  |  5 vs 6 : 6:4  |  7 vs 8 : 4:6  <br>
now playing 1 vs 3  and  5 vs 8


In [49]:
time = 1  #expect about 120secs / 2 mins per game

agent1 = simple_mcts_timelimited_Agent(1, 0.1, time)
agent3 = simple_mcts_timelimited_Agent(1, 0.3, time)
agent5 = simple_mcts_timelimited_Agent(1, 0.5, time)
agent8 = simple_mcts_timelimited_Agent(1, 0.8, time)

players = [(agent1,agent3), (agent5,agent8)]
results = []

for pair in players:
    start = monotonic()
    game = Game(pair[0], pair[1], 10)
    game.run()
    print(game.win_rate())
    end = monotonic()
    minutes = int((end - start)/60)
    print("Time elapsed during the 10 plays (in minutes):", minutes)
    results.append((game.win_rate(),minutes))
 
print(results)

1
2
3
4
5
6
6
7
8
9
0.9
Time elapsed during the 10 plays (in minutes): 8
1
2
3
4
4
5
6
7
8
8
0.8
Time elapsed during the 10 plays (in minutes): 9
[(0.9, 8), (0.8, 9)]


0.9 <br> 
Time elapsed during the 10 plays (in minutes): 8 <br>
0.8 <br>
Time elapsed during the 10 plays (in minutes): 9 <br>

Agent 1 clearly beats agent 3 here

<b> As far as 1 sec per mcts goes, 0.1 is the clear best c </b>



What happens if we switch from 1minute to 5minutes - is the result gonna be the opposite for agent1 and agent2

In [6]:
time = 5  #expect about 600secs / 5 mins per game - (max) more like 10

agent1 = simple_mcts_timelimited_Agent(1, 0.1, time)
agent2 = simple_mcts_timelimited_Agent(1, 0.2, time)

start = monotonic()
# benchmark test - 1 vs 20 default playouts, c = 0.5, 1 sec per mcts
game = Game(agent1, agent2, 6)
game.run()
print(game.win_rate())
end = monotonic()
print("Time elapsed during matches:", end - start)

0.3333333333333333
Time elapsed during matches: 1036.0792464679998


0.33  <br> 
Time elapsed during matches: 1198.0619802670008

So with longer time per search we see a movement towards bigger exploration factor <br>
However between 1 and 5 seconds 0.1 is still close to 0.2

<b> i.e. we see the same trend as for playouts (the more time we have - the more we're moving toward exploration / bigger number), but slower change speed </b>

### 4.2 Sample test for c with resolution .02

In [51]:
time = 1  #expect about 120secs / 2 mins per game

agent1 = simple_mcts_timelimited_Agent(1, 0.06, time)
agent2 = simple_mcts_timelimited_Agent(1, 0.08, time)
agent3 = simple_mcts_timelimited_Agent(1, 0.1, time)
agent4 = simple_mcts_timelimited_Agent(1, 0.12, time)
agent5 = simple_mcts_timelimited_Agent(1, 0.14, time)

players = [(agent1,agent3), (agent2,agent3), (agent4,agent3), (agent5,agent3)]
results = []

for pair in players:
    start = monotonic()
    game = Game(pair[0], pair[1], 10)
    game.run()
    print(game.win_rate())
    end = monotonic()
    minutes = int((end - start)/60)
    print("Time elapsed during the 10 plays (in minutes):", minutes)
    results.append((game.win_rate(),minutes))
 
print(results)

0.3
Time elapsed during the 10 plays (in minutes): 8
0.7
Time elapsed during the 10 plays (in minutes): 9
0.3
Time elapsed during the 10 plays (in minutes): 8
0.4
Time elapsed during the 10 plays (in minutes): 8
[(0.3, 8), (0.7, 9), (0.3, 8), (0.4, 8)]


0.3 <br>
Time elapsed during the 10 plays (in minutes): 8 <br>
0.7 <br>
Time elapsed during the 10 plays (in minutes): 9 <br>
0.3 <br>
Time elapsed during the 10 plays (in minutes): 8 <br>
0.4 <br>
Time elapsed during the 10 plays (in minutes): 8 <br>

So our agent 2 excelled in last tournament (c = 0.08)

Note that we already established that the most effective c depends on the time allocated per mcts.
So more tests should be done only when comparing with the uni machine.

-----------

# General Conclusions #


1. Tweaking parameters (the main playouts and c in this case) seems to be a promising way to improve MCTS.
2. Generally the fewer time there is per mcts the less exploration there must be, and fewer default playouts. But more time - more playouts and bigger c improve win rates.
3. We got a rough idea of where those numbers stand. But they need to be specifically tweaked in regards to the final machine specifications.


In [7]:
time = 5  #expect about 600secs / 5 mins per game - (max) more like 10

agent1 = simple_mcts_timelimited_Agent(1, 0.2, time)
agent2 = simple_mcts_timelimited_Agent(1, 1.2, time)

start = monotonic()
# benchmark test - 1 vs 20 default playouts, c = 0.5, 1 sec per mcts
game = Game(agent1, agent2, 6)
game.run()
print(game.win_rate())
end = monotonic()
print("Time elapsed during matches:", end - start)

0.8333333333333334
Time elapsed during matches: 1101.4862713799998
